In [1]:
# need to use updated master JupiterCVML code
import sys
# sys.path.append('/home/li.yu/code/JupiterCVML/europa/base/src/europa')

import os
os.environ["BRT_ENV"] = 'prod'
import json
import random
import brtdevkit
print(brtdevkit.__version__)
brtdevkit.log = 'info'
import os
os.environ['AWS_PROFILE'] = 'default'
USERNAME = 'li.yu'

import time
import numpy as np
import pandas as pd
from datetime import datetime, timezone

from brtdevkit.core.db.athena import AthenaClient, Table
from brtdevkit.data import Image, Dataset

# from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import *
# from aletheia_dataset_creator.config.dataset_config import *

from jupiterdata.config.dataset_config import *
from jupiterdata.utils.dataset import *

pd.set_option('display.max_columns', None)

# %matplotlib inline
%load_ext autoreload
%autoreload 2

7.16.2


/home/li.yu/anaconda3/envs/brtdevkit/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
""" # run this on cmd: 
eval "$(/home/li.yu/anaconda3/bin/conda shell.bash hook)"
conda activate brtdevkit
brt-devkit-auth
aws sso login --profile jupiter_prod_engineer-425642425116
"""
import inspect
print(inspect.getmodule(Dataset))
print(LEFT_CAMERAS)
print(ALL_CAMERA_PAIRS_LIST)

<module 'brtdevkit.data.core.dataset' from '/home/li.yu/anaconda3/envs/brtdevkit/lib/python3.9/site-packages/brtdevkit/data/core/dataset.py'>
['front-center-left', 'front-left-left', 'front-right-left', 'side-left-left', 'side-right-left', 'rear-left', 'T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14', 'I01', 'I02']
[{'front-center-left': 'front-center-right', 'front-left-left': 'front-left-right', 'front-right-left': 'front-right-right', 'side-left-left': 'side-left-right', 'side-right-left': 'side-right-right', 'rear-left': 'rear-right', 'front-center-right': 'front-center-left', 'front-left-right': 'front-left-left', 'front-right-right': 'front-right-left', 'side-left-right': 'side-left-left', 'side-right-right': 'side-right-left', 'rear-right': 'rear-left'}, {'T01': 'T03', 'T02': 'T04', 'T05': 'T07', 'T06': 'T08', 'T09': 'T11', 'T10': 'T12', 'T13': 'T15', 'T14': 'T16', 'I01': 'I03', 'I02': 'I04'}, {'T02': 'T03', 'T06': 'T07', 'T10': 'T11', 'T14': 'T15', 'I02': 'I03'}]


In [2]:
# test_dataset = Dataset.retrieve(id='64ed2657926aefcd654e8269')
test_dataset = Dataset.retrieve(name='halo_vehicles_in_dust_collection_june07')
print(test_dataset.id)
test_df = test_dataset.to_dataframe()
test_df.shape

6668a51ba11727e13fe9b622


2024-06-12 12:20:45,390 - APIRequestor - ERROR - API error received | error_code : 403, error_message : {'message': 'Role token_exchange is required to make this API call', 'extra': {'role_name': 'token_exchange'}}


(39034, 104)

In [ ]:
test_dfs = []
test_dfs.append(test_df)

In [8]:
test_df[['id', 'camera_location']].groupby('camera_location').size()

camera_location
T01    7690
T02    7696
T03    7693
T04    7696
T05    7695
T06    7697
T07    7689
T08    7696
T09    8249
T10    8252
T11    8250
T12    8254
T13    8251
T14    8252
T15    8254
T16    8254
dtype: int64

## Run Databricks Query

In [9]:
# from jupiterdata.db import databricks_connector
# databricks = databricks_connector.Databricks()
# # note on json extraction https://docs.databricks.com/en/sql/language-manual/sql-ref-json-path-expression.html
# df = databricks.execute(query)
# print(df.shape, len(df)//2)

query = f"""
SELECT
  IMJ.id,
  IMJ.operation_time,
  IMJ.camera_location,
  IMJ.robot_name,
  IMJ.collected_on,
  IMJ.special_notes,
  IMJ.hard_drive_name
FROM
  mesa_prod.mesa_lake_prod.image_jupiter AS IMJ
WHERE
  IMJ.robot_name = 'bedrock_411'
  AND IMJ.hard_drive_name = 'JUPD-0611_2024-6-5'
"""

  # IMJ.has_human_annotation,
  # IMJ.has_nearby_stop_event,
  # IMJ.operating_field_name,
  # IMJ.farm,
  # IMJ.bag_name,
  # IMJ.closest_object_info__json,
  # IMJ.created_at,
  # IMJ.hard_drive_name,
  # IMJ.state,
  # IMJ.hdr_mode,
  # IMJ.group_id,
  # IMJ.jdb_s3_path,
  # IMJ.ros_s3_path,
  # IMJ.bundle,
  # IMJ.gps_can_data__json,
  # IMJ.implement_angle_data__json,
  # IMJ.autonomy_state__json,
  # IMJ.sensor_type,
  # IMJ.geohash,
  # IMJ.calibration_data__json,
  # IMJ.tractor_type
#   IMJ.robot_name = 'bedrock_411'
#   AND IMJ.collected_on > date('2024-04-22')
#   AND IMJ.collected_on <= date('2024-04-24')

# hard_drive_name = 'JUPD-0306_2024-2-28'  # first dust collection
# hard_drive_name = 'JUPD-0325_2024-3-19'  # vehicle collection, not necessarily dust
# https://bluerivertechnology.slack.com/archives/C04UNEQ190W/p1711143867529459?thread_ts=1711142618.986079&cid=C04UNEQ190W
# https://bluerivertechnology.slack.com/archives/C06MB47MWUE/p1711145579254259

# query = """
# SELECT id, image, properties__json FROM annotation_jupiter
# WHERE style = 'categorical'
# AND created_at > date('2024-01-01')
# AND state in ('review', 'ok')
# AND properties__json LIKE '%"title": "human\_pose"%'
# AND kind = 'labelbox'
# """

# # query for categorical labels
# query = f"""
# SELECT image AS id, properties__json, created_at FROM annotation_jupiter
# WHERE style = 'categorical'
# AND state in ('review', 'ok')
# AND label_map_full = '66343128ce27e38fac43fc3e'
# AND kind = 'labelbox'
# AND annotation_jupiter.vendor_metadata__json LIKE '%vendor_project_id": "clvzvij3v02w1070x7p0hdc55%'
# """

# query = f"""
# SELECT ANJ.id as id, ANJ.image as image, ANJ.properties__json as properties__json, ANJ.created_at as created_at
# FROM mesa_prod.mesa_lake_prod.annotation_jupiter as ANJ
# WHERE ANJ.style = 'categorical'
# AND ANJ.created_at > date('2024-01-01')
# AND ANJ.state in ('review', 'ok')
# AND (properties__json LIKE '%"title": "Pose"%' OR properties__json LIKE '%"title": "Human Clothing"%' OR properties__json LIKE '%"title": "Human Occlusion"%') 
# AND (properties__json LIKE '%"title": "human_pose"%' OR properties__json LIKE '%"title": "human_clothing"%' OR properties__json LIKE '%"title": "human_occlusion"%') 
# AND ANJ.kind = 'labelbox'
#   hard_drive_name = 'JUPD-0325_2024-3-19'
#   AND gps_can_data__json IS NOT NULL
#   AND special_notes IS NOT NULL
#   AND SUBSTRING(IMJ.geohash, 1, 6) IN {tuple(geohash6_test_list)}
#   AND IMJ.sensor_type = 'VD6763'
#   AND (IMJ.jdb_s3_path like '%.mcap' OR IMJ.calibration_data__json:serial_number LIKE 'PCE7T_B%')
# """

# query = f"""
#   SELECT img.id, img.collected_on
#   FROM mesa_prod.mesa_lake_prod.image_jupiter as img
#   JOIN mesa_prod.mesa_lake_prod.image_artifact_jupiter AS A
#       ON A.image = img.id
#   WHERE img.teleop_request__json IS NOT NULL
#   AND img.teleop_request__json:original_roi IS NOT NULL
#   AND NOT EXISTS(
#           SELECT * from mesa_prod.mesa_lake_prod.annotation_jupiter as ann WHERE ann.image = img.id
#           AND ann._cls = 'Annotation.CategoricalAnnotation'
#           AND ann.is_active_version = true
#       )
#   LIMIT 10
#   ;"""


from jupiterdata.utils.dataset import query_db
df = query_db(query)
df = df.sort_values('collected_on')
print(df.iloc[0].collected_on, df.iloc[-1].collected_on)
print(df.shape, len(df)//2)

2024-06-05 21:24:03.099000+00:00 2024-06-08 01:19:17.355000+00:00
(86521, 7) 43260


In [10]:
df.special_notes.unique()

array([None, 'JF106', 'JF-89', 'JF-106'], dtype=object)

In [11]:
df2 = df[df.special_notes.isin(['JF106', 'JF-106'])]
# df2 = df[(df.camera_location.isin(['T09', 'T10', 'T11', 'T12'])) & (df.special_notes != 'test') ]
df2.shape

(39034, 7)

In [12]:
df2 = df2.sort_values('collected_on')
print(df2.iloc[0].collected_on, df2.iloc[-1].collected_on)

2024-06-06 01:52:41.707000+00:00 2024-06-07 03:10:21.833000+00:00


In [15]:
df3 = df2[df2.camera_location.isin(['T09', 'T10'])]
df3.shape

(5838, 6)

In [10]:
df.groupby('camera_location').size()

camera_location
T01    6849
T02    6848
T03    6850
T04    6851
T05    6851
T06    6851
T07    6852
T08    6851
T09    6848
T10    6849
T11    6846
T12    6848
T13    6851
T14    6851
T15    6849
T16    6852
dtype: int64

In [5]:
df.camera_location.unique()

array(['T03', 'T02', 'T07', 'T08', 'T05', 'T06', 'T04', 'T01', 'T12',
       'T14', 'T13', 'T15', 'T09', 'T16', 'T11', 'T10'], dtype=object)

In [8]:
df.to_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/all_human_pose_0513_v6_new_metadata_for_1934_images.csv', index=False)

In [36]:
df3 = df[df.collected_on >= '2024-04-23']
df3 = df3.sort_values('collected_on')
print(df3.shape, df3.special_notes.unique())
print(df3.iloc[0].collected_on, df3.iloc[-1].collected_on)

(355193, 27) [None 'dusty' 'dusty headlands' 'FOLD TOWER-FILIP REESE'
 'FOLD TOWER-AUSTIN EIDE' '12.112 Low Sunlight']
2024-04-23 14:15:07.315000+00:00 2024-04-23 23:59:16.571000+00:00


### Search around birds / airborne debris images

In [55]:
df['serial_number'] = df.calibration_data__json.apply(lambda x: json.loads(x).get('serial_number', np.nan))
df['sn7'] = df['serial_number'].apply(lambda s: s[:7])
df['geohash6'] = df['geohash'].apply(lambda s: s[:6])
df['sn7'].unique(), df['geohash6'].unique()

(array(['PCE7T1B', 'PCE7T1A'], dtype=object),
 array(['9z1pg7', '9z1pgk', '9q9d4w', '9ufs6w', '9z1pgj', '9z1pgh',
        '9ufs6y', '9ufs6v', '9z1pg5', '9vg20v', '9ufs6x', '9ufs6z',
        '9ufs6t', '9zmt8e', '7zzzzz', '9zmt8d', '9q9d4z', '9ufsdg',
        '9ufsd7', '9ufsd8', '9q9d4x', '9q9d4y', '9ufsd2', '9ze3m3',
        '9ze3m7', '9ze3m9', '9ze3me', '9ze3m6', '9ze3md', '9ze3mk',
        '9ze3ms', '9zmt8s', '9z1pyy', '9z1pzm', '9z1pzn', '9z1ptw',
        '9z1pzq', '9z1pzj', '9ze3m2', '9yyzmd', '9yyzm8', '9yyzjr',
        '9yyzm9', '9ufsd3', '9ufsd1', '9ufsd6'], dtype=object))

In [57]:
sbdf = df[df.sn7 == 'PCE7T1B']
sbdf.shape

(9020603, 30)

In [56]:
df.to_csv('/data/jupiter/li.yu/data/halo_hard_cases/all_mcap_data_till_0418.csv', index=False)
# df.to_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/all_human_pose_0326_v4.csv', index=False)

In [ ]:
# read in geohash table
geohash_table = '/data/jupiter/li.yu/data/halo_hard_cases/20240328_geohash_table_v8_1_rc.csv'
geohash_df = pd.read_csv(geohash_table)
geohash6_train_list = geohash_df[geohash_df.bucket == 'train'].geohash6.to_list()
geohash6_test_list = geohash_df[geohash_df.bucket == 'test'].geohash6.to_list()
print(geohash_df.shape, len(geohash6_train_list), len(geohash6_test_list))

In [5]:
import ast
root_dir = '/data2/jupiter/datasets/'
# dataset = 'halo_rgb_stereo_train_v6_2'
dataset = 'halo_rgb_stereo_train_v8_1'
csv = os.path.join(root_dir, dataset, 'master_annotations.csv')
converters = {"label_map": ast.literal_eval, "label_counts": ast.literal_eval}
# converters = {}
df = pd.read_csv(csv, converters=converters)

# ab_df = df[df.has_airborne == True]
# print(ab_df.shape, ab_df.iloc[0].geohash)
# ab_df["geohash_6"] = ab_df["geohash"].apply(lambda x: x[:6])

df['Birds'] = df['label_counts'].apply(lambda d: d.get('Birds', 0))
df['has_birds'] = df['Birds'] > 0
bird_df = df[df.has_birds == True]
print(bird_df.shape, bird_df.iloc[0].geohash)
bird_df["geohash_6"] = bird_df["geohash"].apply(lambda x: x[:6])

from jupiterdata.utils.dataset import query_db

(1362, 361) 9xjg9cde9btm


/tmp/ipykernel_103657/3024971100.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bird_df["geohash_6"] = bird_df["geohash"].apply(lambda x: x[:6])


In [7]:
LEFT_CAMERAS = ('T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14', 'I01', 'I02')
geohash_6_limit = 500
geohash_6_expand = 1.5
surround_ids = []
for geohash_6 in bird_df.geohash_6.unique():
    collected_on_days = bird_df[(bird_df.geohash_6 == geohash_6)].collected_on_day.unique()
    for collected_on_day in collected_on_days:
        current = len(bird_df[(bird_df.geohash_6 == geohash_6) & (bird_df.collected_on_day == collected_on_day)].unique_id.unique())
        expand = int(geohash_6_limit / current * geohash_6_expand)
        if expand < 50:
            expand = 50
        elif expand > 500:
            expand = 500

        query = f"""
        SELECT
          IMJ.id AS id,
          IMJ.robot_name AS robot_name,
          IMJ.camera_location AS camera_location,
          IMJ.operating_field_name AS operating_field_name,
          IMJ.bag_name AS bag_name,
          IMJ.collected_on AS collected_on,
          IMJ.hard_drive_name AS hard_drive_name,
          IMJ.geohash AS geohash
        FROM
          mesa_prod.mesa_lake_prod.image_jupiter AS IMJ
        WHERE
          collected_on LIKE '{collected_on_day[:10]}%'
          AND geohash LIKE '{geohash_6}%'
          AND camera_location IN {LEFT_CAMERAS}
        LIMIT {expand};
        """
        df = query_db(query)
        surround_ids += df.id.to_list()
        print(geohash_6, collected_on_day, current, len(df))
    #     break
    # break
    # print('queried geohash_6', geohash_6)

len(surround_ids)

9xjg9c 2023-08-04 13 57
9v5ks7 2023-08-02 153 50
9v5kwy 2023-08-05 2 375
9v5kxj 2023-08-07 20 50
9v5kse 2023-08-02 67 50
9xjg3z 2023-08-05 4 187
9v5kss 2023-08-02 32 50
dp03h5 2023-09-08 3 250
9v5ksk 2023-08-02 22 50
9xjgd0 2023-08-04 4 187
9xjgd0 2023-08-05 1 500
9xjgy5 2023-08-15 8 93
9v5kwu 2023-08-05 9 83
dp03ht 2023-09-09 1 500
9xjg9b 2023-08-04 5 150
9xjg9b 2023-08-05 2 375
9xjg92 2023-07-28 8 93
9xjepp 2023-08-09 2 375
dju21r 2023-08-18 2 375
9v5kxn 2023-08-07 7 107
9xjg3f 2023-08-07 5 150
9xjgy7 2023-08-15 2 375
9xjg8b 2023-08-17 4 187
dju21p 2023-08-18 4 187
9xjg98 2023-07-28 2 375
dp03k9 2023-09-13 3 250
9xjg64 2023-08-07 1 500
9xjg3d 2023-08-07 2 375
9xjgvu 2023-08-15 3 31
9v5kws 2023-08-07 2 375
9v5kwv 2023-08-05 19 50
9xjg9m 2023-08-16 5 150
dp03hx 2023-09-12 4 187
9xjg3r 2023-08-04 1 500
9xjg3r 2023-08-16 1 500
9v5ksd 2023-08-02 5 150
9xjer0 2023-08-09 1 500
dp03k0 2023-09-11 3 250
9v5ks6 2023-08-01 2 375
9v5ks6 2023-08-02 18 50
9xjg6p 2023-08-05 1 500
9xjg3g 2023-08-07 1

60573

In [8]:
remain_ids = list(set(surround_ids) - set(df.id.to_list()))
print(len(remain_ids))
remain_ids[0]

60073


'654658b46c228e535a897327'

### Search around a time

In [3]:
from datetime import datetime, timedelta
from jupiterdata.utils.dataset import query_db

In [36]:
terrain_csv = '/data/jupiter/li.yu/data/dune_simulation/representative_terrains.csv'
terrain_df = pd.read_csv(terrain_csv)
terrain_df['datetime'] = terrain_df.collected_on.apply(datetime.fromisoformat)
delta = timedelta(seconds=2)
print(terrain_df.shape)
terrain_df.head(2)

(15, 5)


,id,collected_on,bag_name,terrain,datetime
0,65eb4802738e53d66c8f346a,2024-02-08T19:22:19.435000,02_08_2024-19_22_27,open_field,2024-02-08 19:22:19.435
1,65fb4c39ae44446ff5d3fc19,2024-02-23T21:18:43.051000,02_23_2024-21_18_45,in_farm,2024-02-23 21:18:43.051


In [40]:
str(terrain_df.iloc[-1].datetime), str(terrain_df.iloc[-1].datetime + delta)[:19]

('2023-10-02 17:39:21.561000', '2023-10-02 17:39:23')

In [37]:
dfs = []
for i,row in terrain_df.iterrows():

    query = f"""
    SELECT
        IMJ.id AS id,
        IMJ.robot_name AS robot_name,
        IMJ.camera_location AS camera_location,
        IMJ.operating_field_name AS operating_field_name,
        IMJ.bag_name AS bag_name,
        IMJ.collected_on AS collected_on,
        IMJ.hard_drive_name AS hard_drive_name,
        IMJ.geohash AS geohash
    FROM
        mesa_prod.mesa_lake_prod.image_jupiter AS IMJ
    WHERE
        bag_name = '{row.bag_name}'
        AND collected_on > '{str(row.datetime - delta)[:19]}'
        AND collected_on <= '{str(row.datetime + delta)[:19]}'
    """

    df = query_db(query)
    print(row.id, row.terrain, len(df))
    dfs.append(df)
    # break

65eb4802738e53d66c8f346a open_field 88
65fb4c39ae44446ff5d3fc19 in_farm 192
65eb1913abb27b5979bb2c49 trees 96
65fb5e03fbbd37ea5e01a225 pole_by_road 191
65fb3f89c7d378dbc65eb080 birds 192
6516a9ee59d4a80830b8f3f2 head_lands 32
64e5d2fd7d3e736bc96a7e64 tall_weeds 80
653b8ab65516ee030864597f irrigation 32
650420e64f21186e0f2bda70 road_side 44
655b1dfb1ec667f4ad177ea1 tall_weeds2 32
65fb3235e9fc675f8fee1f38 dust 192
654d7f53443a59d899337355 flooded_area 60
651f4ca5397e2827e807f262 head_lands2 18
6531b6fa44af03aa185c2c36 head_lands3 48
651f586e7747a76b7ba779a8 open_field2 32


In [38]:
df = df.sort_values('collected_on')
df[['id', 'collected_on']]

,id,collected_on
11,651f53a5d2dd50c6782fc604,2023-10-02 17:39:19.451000+00:00
20,651f53a8d7bd74435539150e,2023-10-02 17:39:19.487000+00:00
4,651f53a6f5298a1f9837649e,2023-10-02 17:39:19.513000+00:00
30,651f53aa4393288ab0595044,2023-10-02 17:39:19.523000+00:00
7,651f53ad1fc961b2c883145a,2023-10-02 17:39:19.528000+00:00
31,651f53b188a7ef1b7c505df2,2023-10-02 17:39:19.538000+00:00
18,651f53af24484d263bf73813,2023-10-02 17:39:19.545000+00:00
12,651f53ab66fd3736ba6d439d,2023-10-02 17:39:19.554000+00:00
5,651f57e52945e575a12a2c3a,2023-10-02 17:39:20.470000+00:00
24,651f57ec234278694e6702c2,2023-10-02 17:39:20.503000+00:00


In [41]:
df = pd.concat(dfs, ignore_index=True)
print(df.shape)

(1329, 8)


### Search around geohash and time

In [7]:
import ast
from datetime import datetime, timedelta
from jupiterdata.utils.dataset import query_db

def get_fn_geohash6_bag_name(root_dir, pred_dir, model, dataset):
    # load dataset df
    csv = os.path.join(root_dir, dataset, 'master_annotations.csv')
    converters = {"label_map": ast.literal_eval, "label_counts": ast.literal_eval}
    df = pd.read_csv(csv, converters=converters)
    df['geohash6'] = df['geohash'].apply(lambda g: g[:6])
    print(df.shape)

    # load pred fns
    sum_pred_dir = os.path.join(pred_dir, model, dataset, 'false_negative_sum5')
    fn_ids = os.listdir(sum_pred_dir)
    unique_ids = [f[:-4] for f in fn_ids]

    # get fn df
    fn_df = df[df.unique_id.isin(unique_ids)]
    return fn_df

In [25]:
# read in geohash table
geohash_table = '/data/jupiter/li.yu/data/halo_hard_cases/20240328_geohash_table_v8_1_rc.csv'
geohash_df = pd.read_csv(geohash_table)
geohash6_train_list = geohash_df[geohash_df.bucket == 'train'].geohash6.to_list()
geohash6_test_list = geohash_df[geohash_df.bucket == 'test'].geohash6.to_list()
print(geohash_df.shape, len(geohash6_train_list), len(geohash6_test_list))

(2101, 2) 1672 371


In [8]:
mcsv = '/data2/jupiter/datasets/halo_rgb_stereo_train_v8_1/master_annotations.csv'
mdf = pd.read_csv(mcsv)
print(mdf.shape)

root_dir = '/data/jupiter/datasets/safety_datasets/'
pred_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model'
model = 'v81_nextvitsmall_dust_only_kore_dean_0406'
datasets = [
    "humans/on_path_forward/on_path_forward_humans_night_2024_rev2_v10", 
    "humans/on_path_aft/on_path_aft_humans_night_2024_rev2_v10",
]
fn_dfs = [get_fn_geohash6_bag_name(root_dir, pred_dir, model, dataset) for dataset in datasets]
fn_df = pd.concat(fn_dfs, ignore_index=True)
print(fn_df.shape)
print(len(fn_df.bag_name.unique()), len(fn_df.geohash6.unique()))

(326871, 359)


/tmp/ipykernel_121511/3595870924.py:9: DtypeWarning: Columns (23,37,132,150,168,184,188,196,218,219) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv, converters=converters)


(4251, 332)
(2260, 332)
(651, 333)
187 23


/tmp/ipykernel_121511/3595870924.py:9: DtypeWarning: Columns (164,165,172,174,184,213,218,219,222,224,239,273,274) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv, converters=converters)


In [9]:
g = fn_df.groupby(['geohash6', 'bag_name']).size()
keys = g.index
d = [[k, g[k]] for k in keys]
d = sorted(d, key=lambda k: k[1], reverse=True)
d[:5]

[[('9v6tpg', '02_14_2024-00_50_50'), 27],
 [('9zpund', '10_21_2023-00_44_44'), 17],
 [('9v6tps', '02_21_2024-01_24_00'), 16],
 [('9zsx7q', '11_06_2023-23_46_58'), 15],
 [('9v6tpg', '02_17_2024-01_35_37'), 14]]

In [14]:
# get collection time of fn images in a bag
def get_start_end_time_of_bag(fn_df, bag_name):
    # bag_name = '02_21_2024-01_24_00'
    bag_df = fn_df[fn_df.bag_name == bag_name]
    bag_df = bag_df.sort_values('collected_on')
    bag_df['datetime'] = bag_df.collected_on.apply(datetime.fromisoformat)
    delta = timedelta(seconds=60*30)
    # print(bag_df.shape)
    # print(bag_df.iloc[0].datetime, bag_df.iloc[-1].datetime)
    # print(bag_df.iloc[0].datetime - delta, bag_df.iloc[-1].datetime + delta)
    return bag_df.iloc[0].datetime - delta, bag_df.iloc[-1].datetime + delta

In [19]:
dfs = {}
for (geohash6, bag_name), N in d:
    t1, t2 = get_start_end_time_of_bag(fn_df, bag_name)

    query = f"""
    SELECT
        IMJ.id AS id,
        IMJ.robot_name AS robot_name,
        IMJ.camera_location AS camera_location,
        IMJ.operating_field_name AS operating_field_name,
        IMJ.bag_name AS bag_name,
        IMJ.collected_on AS collected_on,
        IMJ.hard_drive_name AS hard_drive_name,
        IMJ.geohash AS geohash
    FROM
        mesa_prod.mesa_lake_prod.image_jupiter AS IMJ
    WHERE
        bag_name = '{bag_name}'
        AND collected_on > '{str(t1)[:19]}'
        AND collected_on <= '{str(t2)[:19]}'
    """

    df = query_db(query)
    print(geohash6, bag_name, t1, t2, len(df))
    dfs[(geohash6, bag_name)] = df
    # break

9v6tpg 02_14_2024-00_50_50 2024-02-14 00:21:43.138000 2024-02-14 01:22:19.165000 2078
9zpund 10_21_2023-00_44_44 2023-10-21 00:15:52.182000 2023-10-21 01:16:38.163000 921
9v6tps 02_21_2024-01_24_00 2024-02-21 00:54:22.037000 2024-02-21 01:54:43.033000 3377
9zsx7q 11_06_2023-23_46_58 2023-11-06 23:17:33.235000 2023-11-07 00:18:47.270000 1048
9v6tpg 02_17_2024-01_35_37 2024-02-17 01:06:00.170000 2024-02-17 02:07:00.095000 1616
9v6tpg 02_21_2024-01_02_22 2024-02-21 00:33:58.630000 2024-02-21 01:34:45.661000 2062
9v6tpg 02_24_2024-00_54_03 2024-02-24 00:24:42.477000 2024-02-24 01:25:06.534000 1459
9zmjkf 11_18_2023-02_58_58 2023-11-18 02:30:22.710000 2023-11-18 03:30:47.680000 960
9zmjkf 11_17_2023-23_45_45 2023-11-17 23:16:29.858000 2023-11-18 00:17:37.874000 921
9v6tpg 02_25_2024-01_27_28 2024-02-25 00:58:04.455000 2024-02-25 01:58:22.454000 959
9v6tpg 02_14_2024-00_52_52 2024-02-14 00:23:13.958000 2024-02-14 01:23:32.119000 1182
9v6tpg 02_14_2024-03_00_10 2024-02-14 02:30:58.970000 2024

In [40]:
total = 0
use_df = None
for (geohash6, bag_name), df in dfs.items():
    df['geohash6'] = df['geohash'].apply(lambda g: g[:6])
    # _df = df[(df.geohash6 != '7zzzzz') & (~df.geohash6.isin(geohash6_test_list))]
    _df = df[(~df.geohash6.isin(geohash6_test_list))]
    if len(_df) > 0:
        use_df = _df if len(_df) > 200 else use_df
        total += len(_df)
        print(geohash6, bag_name, _df.shape)
        print(_df.sample(min(5, len(_df))).id.to_list())
        # if len(_df[_df.id == '654bdc10851dd6051c9a2d4b']) > 0:
        #     print(_df[_df.id == '654bdc10851dd6051c9a2d4b'].geohash.to_list())
print(total)

9v6tpg 02_14_2024-00_50_50 (554, 9)
['65ea454263edec78ede83128', '65ea42b9936e32acd000c032', '65ea4a7c000669bb859209b4', '65ea482b71e1684babf8f77f', '65ea462bbf9244df1b4d852e']
9v6tps 02_21_2024-01_24_00 (904, 9)
['65e9cdab3de3d70fe449654b', '65ea0b36000669bb8591757d', '65e9d226e879f487a282a835', '65e9d004cafc4e811b02af3d', '65e9d12bf2f15c5f2ceaa879']
9zsx7q 11_06_2023-23_46_58 (88, 9)
['654bdc741094059fbafd8f00', '654bdc6334b229e20d4375a9', '654bdc2200210142b9092ba1', '654bdbd9fa83693bde13c334', '654bdc32f99b22029bd77dbb']
9v6tpg 02_17_2024-01_35_37 (445, 9)
['65e9c01ebc620c2c14fe4026', '65e9c0945e301af12dab0518', '65e9c12a2bbbdbae66cedae3', '65e9bdb5f04f29f44a32c6ab', '65e9c0784875033da61a0078']
9v6tpg 02_21_2024-01_02_22 (539, 9)
['65ea06aedb75a515c2d7ad48', '65e9ffc6459275487d44b70b', '65ea00ca6592f96a05175cee', '65ea01c0d80ab49742253a49', '65ea06f706198505e2a64568']
9v6tpg 02_24_2024-00_54_03 (396, 9)
['65e7c3694875033da61782fb', '65e7c80ddc04dc9d140eb419', '65e7c5a94875033da61787

In [41]:
use_df.geohash6.unique()

array(['7zzzzz'], dtype=object)

## Create a dataset

In [24]:
# df = pd.read_csv('/data/jupiter/li.yu/data/Jupiter_train_v6_2/train_v6_2_categorical_count.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_hard_cases/20231219_20240119_halo_rgb_left_airborne.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_hard_cases/human_behind_implement_0613.csv')
df = pd.read_csv('/data/jupiter/li.yu/data/halo_hard_cases/halo_human_in_dust_daynightsplit_for_july01.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_hard_cases/halo_vehicle_in_dust_daynightsplit_for_july01.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_hard_cases/halo_vehicle_in_dust_daynightsplit_excludebadnight_for_july01.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/rev1_test_missing_human_pose.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/rev1_test_missing_human_clothing.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/rev1_test_humanv15_day_front_standing_ids.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/train_v8_0_miss_human_pose.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/test_v8_0_miss_human_pose.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/train_v8_1_potential_airborne_birds.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/rev1_test_humanv15_to_fully_label.csv')
# df = pd.read_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/test_v12_miss_human_pose.csv')
# df = pd.read_csv('/data/jupiter/datasets/halo_failure_case_of_box_in_dust/selected_for_label.csv')
# df = pd.read_csv('/data/jupiter/datasets/halo_vehicles_in_dust_collection_march2024/selected_for_missing_label.csv')
# df = pd.read_csv('/data/jupiter/datasets/halo_human_in_dust_day_collection_may29/selected_for_label.csv')
# df = pd.read_csv('/data/jupiter/datasets/halo_human_in_dust_night_collection_june03_2/selected_for_label.csv')
# df = pd.read_csv('/data/jupiter/datasets/halo_human_in_dust_dusk_collection_front_june07/selected_for_label.csv')
# df = pd.read_csv('/data/jupiter/datasets/halo_vehicles_in_dust_collection_june06/selected_for_label.csv')
# df = pd.read_csv('/data/jupiter/datasets/halo_vehicles_in_dust_collection_june07/selected_for_label.csv')
print(df.shape, 'id' in df, 'unique_id' in df)

(126399, 3) True False


In [25]:
df.day_night_split.unique()

array(['day', 'dawn_dusk', 'night'], dtype=object)

In [35]:
df2 = df[df.day_night_split == 'night']
print(df2.shape)

(13217, 3)


In [19]:
# unlabeled dataset
Dataset.create(
    name="halo_vehicles_in_dust_collection_june07_for_partial_vehicle_label_2",
    description="14354 out of 39034 images collected for vehicle in dust and selected for vehicle labeling, at June 06-07, for dusk / night time",
    kind='image',  # annotation or image
    image_ids=df.id.to_list())

{}

In [36]:
# labeled, stereo dataset
imageids_to_dataset(
    image_ids=df2.id.to_list(),
    dataset_name="halo_human_in_dust_night_for_july01",
    dataset_description="human in dust images night time, out of 13217 images",
    dataset_kind='pixelwise_annotation',  # image or categorical_annotation or pixelwise_annotation
    mode='stereo',  # stereo or mono
    production_dataset=False,
)

Warning 9988 images do not have a corresponding annotation.
Preparing stereo dataframe for {'T01': 'T03', 'T02': 'T04', 'T05': 'T07', 'T06': 'T08', 'T09': 'T11', 'T10': 'T12', 'T13': 'T15', 'T14': 'T16', 'I01': 'I03', 'I02': 'I04'}...
Size of left dataframe: 3229
Size of stereo dataframe: 3220
Preparing stereo dataframe for {'T02': 'T03', 'T06': 'T07', 'T10': 'T11', 'T14': 'T15', 'I02': 'I03'}...
Size of left dataframe: 2251
Size of stereo dataframe: 2250
Sending 3228 annotated_ids for creating dataset
Time taken to prepare data for dataset creation job: 0.62 mins


In [7]:
# imageids_to_dataset(
#     image_ids=test_df[test_df.camera_location.str.endswith('left')].id.to_list(),
#     dataset_name="20230823_labeled_right_images_12k_left_images_labels",
#     dataset_description="left images and labels of 20230823_labeled_right_images_12k",
#     dataset_kind='annotation',  # annotation or image
#     mode='mono',  # stereo or mono
#     camera_location=CORE_LEFT_CAMERAS,  # CORE_LEFT_CAMERAS or CORE_RIGHT_CAMERAS or ALL_CORE_CAMERAS
# )
# imageids_to_dataset(
#     image_ids=test_df[test_df.camera_location.str.endswith('right')].id.to_list(),
#     dataset_name="20230823_labeled_right_images_12k_right_images_labels",
#     dataset_description="right images and labels of 20230823_labeled_right_images_12k",
#     dataset_kind='annotation',  # annotation or image
#     mode='mono',  # stereo or mono
#     camera_location=CORE_RIGHT_CAMERAS,  # CORE_LEFT_CAMERAS or CORE_RIGHT_CAMERAS or ALL_CORE_CAMERAS
# )

## Download a dataset

In [45]:
# to download large image dataset and save in jpg format, change the following files
# note original files are ending with .bk, and changed files are ending with .save_jpg
# /home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/brtdevkit/data/core/dataset.py
# /home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/brtdevkit/util/aws/s3.py
# /home/li.yu/anaconda3/envs/brtdevkit/lib/python3.9/site-packages/brtdevkit/data/core/dataset.py  # save only one png file

In [3]:
dataset_name = 'halo_human_in_dust_day_collection_may29'
dataset_dir = os.path.join('/data/jupiter/datasets', dataset_name)
# dataset_dir = os.path.join('/data2/jupiter/datasets', dataset_name)
# dataset_dir = os.path.join('/data/jupiter/li.yu/data', dataset_name)
# dataset_dir = os.path.join('/data/jupiter/datasets/image_quality_datasets', dataset_name)
os.makedirs(dataset_dir, exist_ok=True)

test_dataset = Dataset.retrieve(name=dataset_name)
test_df = test_dataset.to_dataframe()
test_dataset.download(dataset_dir, df=test_df)

In [6]:
[f'T{str(i+1).zfill(2)}' for i in range(16)][12:16]

['T13', 'T14', 'T15', 'T16']